# 🌿 AyurVeda Home Remedies Assistant
### Domain-Specific LLM using OpenRouter + LangChain
> **Reference:** *The Complete Book of Ayurvedic Home Remedies* — Vasant Lad

---
### 📋 What this notebook does:
- Connects to **OpenRouter API** (free LLM access)
- Uses **LangChain** for prompt engineering & chain building
- Answers **ONLY** Ayurvedic home remedy questions
- **Refuses** all out-of-domain queries politely
- Runs an **interactive chat widget** right inside Colab

---
### ⚠️ Before You Start:
1. Get a **free API key** from [openrouter.ai/keys](https://openrouter.ai/keys)
2. Run each cell **in order** (Shift+Enter)
3. Paste your API key when prompted

---
## 📦 Step 1 — Install Dependencies

In [ ]:
!pip install -q langchain langchain-openai openai python-dotenv
print('✅ All packages installed!')

---
## 🔑 Step 2 — Configure API Key

**To use Colab Secrets (safest):**
1. Click the 🔑 key icon in the left sidebar
2. Click `+ Add new secret`
3. Name: `OPENROUTER_API_KEY`, Value: your key
4. Toggle it **ON**

In [ ]:
import os
from getpass import getpass

try:
    from google.colab import userdata
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    print('✅ API key loaded from Colab Secrets!')
except Exception:
    print('Colab Secrets not found. Please enter your key below.')
    print('Get a free key at: https://openrouter.ai/keys')
    OPENROUTER_API_KEY = getpass('🔑 Enter your OpenRouter API Key: ')
    print('✅ API key received!')

if not OPENROUTER_API_KEY:
    raise ValueError('❌ No API key provided!')

os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY
print(f'🔒 Key secured (ends with: ...{OPENROUTER_API_KEY[-4:]})')

---
## 🤖 Step 3 — Initialise the LLM via OpenRouter

In [ ]:
from langchain_openai import ChatOpenAI

OPENROUTER_BASE_URL = 'https://openrouter.ai/api/v1'
MODEL_NAME = 'mistralai/mistral-small-3.1-24b-instruct:free'

def get_llm(temperature=0.4):
    return ChatOpenAI(
        model=MODEL_NAME,
        openai_api_key=os.environ['OPENROUTER_API_KEY'],
        openai_api_base=OPENROUTER_BASE_URL,
        temperature=temperature,
    )

# Quick test
test_llm = get_llm()
test_response = test_llm.invoke('Say Namaste in one word only.')
print(f'✅ LLM connected! Test: {test_response.content.strip()}')

---
## 📝 Step 4 — System Prompt (Prompt Engineering)
This defines role, domain scope, output format, tone, disclaimer, and refusal behavior.

In [ ]:
SYSTEM_PROMPT = """
You are AyurVeda Assistant — a friendly, knowledgeable Ayurvedic Home Remedy advisor.
Your knowledge is strictly based on classical Ayurvedic principles as described in
The Complete Book of Ayurvedic Home Remedies by Vasant Lad.

ROLE AND EXPERTISE
- Expert in Vata, Pitta, Kapha doshas, Prakriti, Agni, Ama.
- Knowledgeable about herbs: ashwagandha, triphala, tulsi, neem, ginger, turmeric.
- Guides users through food, herbs, and daily habits.

DOMAIN BOUNDARIES (what you WILL answer)
- Common ailments: cold, indigestion, insomnia, stress, skin issues, joint pain
- Dosha identification and balancing
- Ayurvedic diet and food recommendations
- Herbal preparations and their uses
- Ayurvedic daily and seasonal routines
- General Ayurvedic concepts and philosophy

OUT-OF-DOMAIN TOPICS (REFUSE these politely)
- Allopathic medicine, prescriptions, antibiotics
- Surgical procedures or emergency medical advice
- Mental health clinical therapy
- Financial, legal, or coding queries
- Any topic unrelated to Ayurvedic home remedies

If a query is OUT OF DOMAIN:
- Politely explain you only cover Ayurvedic home remedies
- Suggest appropriate resources
- Offer to help with an Ayurvedic question instead
- Do NOT use the 5-section format for refusals

MANDATORY OUTPUT FORMAT (for all IN-DOMAIN queries)

**Ayurvedic Perspective:**
[Explain the Ayurvedic view of the condition in 2-3 sentences]

**Dosha Involvement:**
[Identify Vata / Pitta / Kapha and explain why]

**Recommended Home Remedies:**
[List 2-4 specific, actionable remedies with instructions]

**Dietary Suggestions:**
[Foods to favour and foods to avoid]

**Disclaimer:**
These are traditional Ayurvedic home remedies for general wellness only.
They are NOT a substitute for professional medical advice.
Please consult a qualified healthcare provider for serious conditions.

TONE
- Warm, supportive, and educational.
- Briefly explain Sanskrit terms when used.
- Never diagnose; always say Ayurveda suggests or traditionally used for.
"""

print('✅ System prompt defined!')
print(f'Prompt length: {len(SYSTEM_PROMPT)} characters')

---
## ⛓️ Step 5 — Build the LangChain LCEL Chain

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

def get_prompt_template():
    return ChatPromptTemplate.from_messages([
        ('system', SYSTEM_PROMPT),
        ('human', '{user_query}'),
    ])

def build_chain(temperature=0.4):
    llm    = get_llm(temperature)
    prompt = get_prompt_template()
    parser = StrOutputParser()
    return prompt | llm | parser

def ask_ayurveda(query, temperature=0.4):
    chain = build_chain(temperature)
    return chain.invoke({'user_query': query})

print('✅ Chain built: ChatPromptTemplate | ChatOpenAI | StrOutputParser')

---
## 🧪 Step 6 — Quick Single Query Test

In [ ]:
from IPython.display import Markdown, display

test_query = 'What are Ayurvedic remedies for indigestion and bloating?'
print(f'Query: {test_query}')
print('Calling OpenRouter API...\n')

response = ask_ayurveda(test_query)
display(Markdown(response))

---
## 🧾 Step 7 — Full Test Suite (11 Queries)
8 in-domain + 3 out-of-domain refusal tests. Results saved to `sample_outputs.txt`.

In [ ]:
import time

TEST_QUERIES = [
    {'id': 1,  'cat': 'IN-DOMAIN',     'q': 'What are Ayurvedic remedies for the common cold and runny nose?'},
    {'id': 2,  'cat': 'IN-DOMAIN',     'q': 'How can I balance Vata dosha through diet and lifestyle?'},
    {'id': 3,  'cat': 'IN-DOMAIN',     'q': 'What Ayurvedic herbs help with insomnia and poor sleep?'},
    {'id': 4,  'cat': 'IN-DOMAIN',     'q': 'I have frequent indigestion and bloating. What does Ayurveda recommend?'},
    {'id': 5,  'cat': 'IN-DOMAIN',     'q': 'What is Triphala and how should I use it?'},
    {'id': 6,  'cat': 'IN-DOMAIN',     'q': 'What Ayurvedic remedies are good for dry skin and eczema?'},
    {'id': 7,  'cat': 'IN-DOMAIN',     'q': 'How does Ayurveda suggest managing stress and anxiety naturally?'},
    {'id': 8,  'cat': 'IN-DOMAIN',     'q': 'What is Dinacharya and what does an Ayurvedic morning routine look like?'},
    {'id': 9,  'cat': 'OUT-OF-DOMAIN', 'q': 'What is the best antibiotic for a bacterial chest infection?'},
    {'id': 10, 'cat': 'OUT-OF-DOMAIN', 'q': 'Can you help me write a Python function to sort a list?'},
    {'id': 11, 'cat': 'OUT-OF-DOMAIN', 'q': 'What stocks should I invest in this year for maximum returns?'},
]

results = []
output_text = 'AYURVEDA ASSISTANT - SAMPLE TEST OUTPUTS\n' + '='*70 + '\n\n'

for t in TEST_QUERIES:
    print(f"\n{'='*60}")
    print(f"Test {t['id']} | {t['cat']}")
    print(f"Query: {t['q']}")
    print('-'*60)
    resp = ask_ayurveda(t['q'])
    display(Markdown(resp))
    results.append({**t, 'response': resp})
    output_text += f"Test {t['id']} | {t['cat']}\nQuery: {t['q']}\n{'-'*60}\n{resp}\n\n" + '='*70 + '\n\n'
    time.sleep(0.8)

with open('sample_outputs.txt', 'w', encoding='utf-8') as f:
    f.write(output_text)

print('\n✅ All tests complete! Results saved to sample_outputs.txt')

---
## 💾 Step 8 — Download sample_outputs.txt

In [ ]:
from google.colab import files
files.download('sample_outputs.txt')
print('✅ Download started!')

---
## 💬 Step 9 — Interactive Chat Widget

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

title = widgets.HTML(value='<h3>🌿 AyurVeda Home Remedies Assistant</h3><hr>')

temp_slider = widgets.FloatSlider(
    value=0.4, min=0.0, max=1.0, step=0.1,
    description='Temp:',
    layout=widgets.Layout(width='400px')
)

query_input = widgets.Textarea(
    placeholder='Ask an Ayurvedic question... (e.g. What helps with joint pain?)',
    layout=widgets.Layout(width='650px', height='80px')
)

ask_btn   = widgets.Button(description='Ask AyurVeda', button_style='success',
                            layout=widgets.Layout(width='160px', height='40px'))
clear_btn = widgets.Button(description='Clear', button_style='warning',
                            layout=widgets.Layout(width='100px', height='40px'))

output_area = widgets.Output()
history = []

def on_ask(b):
    query = query_input.value.strip()
    if not query:
        return
    with output_area:
        clear_output(wait=True)
        display(Markdown('*Consulting Ayurvedic wisdom...*'))
    response = ask_ayurveda(query, temperature=temp_slider.value)
    history.append((query, response))
    with output_area:
        clear_output(wait=True)
        for i, (q, r) in enumerate(reversed(history)):
            display(Markdown(f'**You (Q{len(history)-i}):** {q}'))
            display(Markdown('---'))
            display(Markdown(r))
            display(Markdown('<br>'))

def on_clear(b):
    history.clear()
    query_input.value = ''
    with output_area:
        clear_output()

ask_btn.on_click(on_ask)
clear_btn.on_click(on_clear)

display(widgets.VBox([
    title,
    widgets.HTML('<b>Temperature (0=precise, 1=creative):</b>'),
    temp_slider,
    widgets.HTML('<b>Your Question:</b>'),
    query_input,
    widgets.HBox([ask_btn, clear_btn]),
    widgets.HTML('<hr>'),
    output_area
]))

---
## 📊 Step 10 — Project Summary

In [ ]:
summary = """
## AyurVeda Assistant - Project Summary

| Component | Details |
|---|---|
| **Domain** | Ayurvedic Home Remedies (Vasant Lad) |
| **LLM Gateway** | OpenRouter API (free tier) |
| **Model** | mistralai/mistral-7b-instruct |
| **Framework** | LangChain LCEL chain |
| **Chain** | ChatPromptTemplate | ChatOpenAI | StrOutputParser |
| **Prompt Layers** | Role, Domain Scope, Output Format, Tone, Disclaimer, Refusal |
| **In-Domain** | Doshas, herbs, diet, remedies, Dinacharya |
| **Out-of-Domain** | Modern medicine, coding, finance - polite refusal |
| **Output Format** | 5 fixed sections every response |
| **Test Cases** | 8 in-domain + 3 out-of-domain = 11 total |
"""
display(Markdown(summary))